<a href="https://colab.research.google.com/github/ilnegri7/prova/blob/main/Copy_of_test_standard_deviation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lucio: Con questo notebook voglio vedere come la funzione scipy.optimize.curve_fit calcola la deviazione standard

(per favore fate una copia sul vostro drive così ognuno ha la propria)

In [ ]:
import numpy as np
import scipy.optimize as sop
import statistics

Definisco 10 numeri a caso attorno a 1, poi ne prendo i primi "ndata"

In [ ]:
x=[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]                       # questo è un array fittizio, serve solo per la lista dei parametri di curve_fit
y=[1.2, 0.8, 1.2, 0.8, 1.2, 0.8, 1.2, 0.8, 1.2, 0.8]    # questi sono i dati di cui voglio trovare media e deviazione standard

ndata=8
x1=np.array(x[0:ndata])
y1=np.array(y[0:ndata])
#print('x1=',x1)
print('y1=',y1)

y1= [1.2 0.8 1.2 0.8 1.2 0.8 1.2 0.8]


Come prima cosa calcolo media e deviazione standard con le funzioni di statistics

In [ ]:
average=statistics.mean(y1)
st_dev =statistics.stdev(y1)
print('average, standard deviation=',average,st_dev)

average, standard deviation= 1.0 0.21380899352993946


**Lucio:** Ora calcolo la deviazione standard $\sigma=\sqrt{\frac{1}{n-1}\sum_j (y_j-\bar{y})^2}$ a mano, per sicurezza: e torna con il valore di prima

In [ ]:
variance=0.
for i in range(0,ndata):
  variance=variance + ((y[i]-average)**2) / (ndata-1)
st_dev_check=np.sqrt(variance)
print('st_dev_check=', st_dev_check , "\t var= ", variance)
print('st_dev_media=', st_dev_check/np.sqrt(ndata))

st_dev_check= 0.21380899352993948 	 var=  0.0457142857142857
st_dev_media= 0.07559289460184543


Oppure, rimaneggiando la formula per fare sigma e mean "running"
$\sigma = \sqrt{\frac{N \sum_j y_j^2 - (\sum_j y_j)^2 }{N-1}}$

In [ ]:
sum = 0.
sum2= 0.
for i in range(0,ndata):
  sum2 = sum2 + y[i]*y[i]
  sum  = sum + y[i]
print("Sum =", sum)
print("Sum2=", sum2)
print("Avg =", sum/ndata)
sig = np.sqrt( (ndata*sum2 - sum*sum))/(ndata-1) 
print("Sig =", sig)

Sum = 8.0
Sum2= 8.32
Avg = 1.0
Sig = 0.22857142857142868


Adesso definisco la funzione di fit, ossia una costante: x è la variabile indipendente, ave è il parametro di fit

In [ ]:
def fun(x,ave):
  return ave

Poi fitto i dati con curve_fit, vedi la documentazione qui https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

La documentazione dice "*To compute one standard deviation errors on the parameters use perr = np.sqrt(np.diag(pcov))*"...

In [ ]:
popt,pcov=sop.curve_fit(fun,x1,y1)
#print('popt,pcov=',yopt[0],ycov[0])
st_dev_1=np.sqrt(      pcov[0])
st_dev_2=np.sqrt(ndata*pcov[0])
print('popt,st_dev_1,st_dev_2=',popt[0],st_dev_1[0],st_dev_2[0])

popt,st_dev_1,st_dev_2= 1.0485714285715344 0.07420783505011162 0.19633547687509773


In [ ]:
print(pcov)
print(np.diag(pcov))

[[0.0055068]]
[0.0055068]


... ma ecco il problema: per ritrovare la deviazione standard di prima, non basta calcolare np.sqrt(pcov[0]), bisogna calcolare invece np.sqrt(ndata*pcov[0]). Naturalmente si trova lo stesso con tutti i valori di ndata. C'è qualcosa di inesatto nella documentazione? O sono io che non ho capito?

In [ ]:
!pip install lmfit

     |████████████████████████████████| 266kB 13.3MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 256kB 17.9MB/s 
  Created wheel for lmfit: filename=lmfit-1.0.1-cp36-none-any.whl size=81989 sha256=89a8bdc0d0c3964d79ac2b8ab5de9a5e90cfc4bdef58d5eae71d2fd09900f7bc
  Stored in directory: /root/.cache/pip/wheels/ef/22/8d/6cd9c0af36f501cfb248673b05dad5d2c27504951d55519acb
  Created wheel for asteval: filename=asteval-0.9.21-cp36-none-any.whl size=17286 sha256=3656d28aea9be30815d33f05f403042e77133489df08f87b5393461094907310
  Stored in directory: /root/.cache/pip/wheels/53/9d/3b/da1ff15feb7d97bb195837f4e78f971c748a945bd4720003ce
Successfully built lmfit asteval


In [ ]:
from lmfit import Minimizer, Parameters, report_fit
import numpy as np
params = Parameters()
params.add('m', value=1)

In [ ]:
def fcn2min(params, t, data):
  m = params['m']
  model = m
  return model - data

In [ ]:
minner = Minimizer(fcn2min, params, fcn_args=(x1, y1))
result = minner.minimize()
final = x1 + result.residual
report_fit(result)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 2
    # data points      = 8
    # variables        = 1
    chi-square         = 0.32000000
    reduced chi-square = 0.04571429
    Akaike info crit   = -23.7510066
    Bayesian info crit = -23.6715651
[[Variables]]
    m:  1.00000000 +/- 0.07559289 (7.56%) (init = 1)


In [ ]:
cov=result.covar
st_dev_2=np.sqrt(ndata*cov[0])
print(st_dev)

0.21380899352993946
